In [ ]:
import pandas as pd
df_train=pd.read_csv('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/Train.csv')
df_test=pd.read_csv('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/Test.csv')
df_val=pd.read_csv('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/Validation.csv')
df_train

,Unnamed: 0,index,main_img_url,text,title,type
0,0,1,https://static01.nyt.com/images/2016/11/23/us/...,"They stood in line at Trump Tower, sometimes u...","At Donald Trump<U+2019>s Properties, a Showcas...",real
1,1,2,https://static01.nyt.com/images/2016/10/18/us/...,Donald J. Trump <U+2019>s foundation informed ...,Trump Foundation Tells New York It Has Stopped...,real
2,2,3,https://static01.nyt.com/images/2016/11/12/us/...,President-elect Donald J. Trump won the White ...,"Donald Trump Prepares for White House Move, bu...",real
3,3,4,https://static01.nyt.com/images/2016/10/21/bus...,An investment pitch for a new Texas hotel is t...,Luring Chinese Investors With Trump<U+2019>s N...,real
4,4,5,https://static01.nyt.com/images/2016/11/21/us/...,President-elect Donald J. Trump <U+2019>s wife...,Melania and Barron Trump Won<U+2019>t Immediat...,real
...,...,...,...,...,...,...
2831,11598,11599,http://www.pravda.ru/image/article/5/9/2/35459...,0 <U+043A><U+043E><U+043C><U+043C><U+0435><U+0...,<U+041F><U+0443><U+0442><U+0438><U+043D> <U+04...,fake
2832,11599,11600,http://www.pravda.ru/image/preview/article//9/...,0 <U+043A><U+043E><U+043C><U+043C><U+0435><U+0...,<U+0423><U+0447><U+0435><U+043D><U+044B><U+043...,fake
2833,11600,11601,http://www.pravda.ru/image/preview/article//9/...,<U+041C><U+0438><U+0440> <U+00BB> <U+0410><U+0...,<U+041A><U+0443><U+0440><U+0434><U+044B> <U+04...,fake
2834,11601,11602,http://www.pravda.ru/image/article/5/8/9/35458...,0 <U+043A><U+043E><U+043C><U+043C><U+0435><U+0...,<U+041F><U+0440><U+0435><U+0437><U+0438><U+043...,fake


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2836 entries, 0 to 2835
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    2836 non-null   int64 
 1   index         2836 non-null   int64 
 2   main_img_url  2836 non-null   object
 3   text          2836 non-null   object
 4   title         2836 non-null   object
 5   type          2836 non-null   object
dtypes: int64(2), object(4)
memory usage: 133.1+ KB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    997 non-null    int64 
 1   index         997 non-null    int64 
 2   main_img_url  997 non-null    object
 3   text          997 non-null    object
 4   title         997 non-null    object
 5   type          997 non-null    object
dtypes: int64(2), object(4)
memory usage: 46.9+ KB


In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    526 non-null    int64 
 1   index         526 non-null    int64 
 2   main_img_url  526 non-null    object
 3   text          526 non-null    object
 4   title         526 non-null    object
 5   type          526 non-null    object
dtypes: int64(2), object(4)
memory usage: 24.8+ KB


In [ ]:
X_train=df_train.drop(['Unnamed: 0','main_img_url','type'],axis=1)
X_test=df_test.drop(['Unnamed: 0','main_img_url','type'],axis=1)
X_val=df_val.drop(['Unnamed: 0','main_img_url','type'],axis=1)
X_train.head(5)

,index,text,title
0,1,"They stood in line at Trump Tower, sometimes u...","At Donald Trump<U+2019>s Properties, a Showcas..."
1,2,Donald J. Trump <U+2019>s foundation informed ...,Trump Foundation Tells New York It Has Stopped...
2,3,President-elect Donald J. Trump won the White ...,"Donald Trump Prepares for White House Move, bu..."
3,4,An investment pitch for a new Texas hotel is t...,Luring Chinese Investors With Trump<U+2019>s N...
4,5,President-elect Donald J. Trump <U+2019>s wife...,Melania and Barron Trump Won<U+2019>t Immediat...


In [ ]:
category={'fake':0,'real':1}
y_train=df_train['type'].map(category)
y_test=df_test['type'].map(category)
y_val=df_val['type'].map(category)
y_train.head(5)

0    1
1    1
2    1
3    1
4    1
Name: type, dtype: int64

In [ ]:
y_test.value_counts(),y_train.value_counts(),y_val.value_counts()

(1    541
 0    456
 Name: type, dtype: int64, 1    1603
 0    1233
 Name: type, dtype: int64, 0    293
 1    233
 Name: type, dtype: int64)

In [ ]:
messages_train=X_train.copy()
messages_test=X_test.copy()
messages_val=X_val.copy()
messages_train.title[0]

'At Donald Trump<U+2019>s Properties, a Showcase for a Brand and a President-Elect'

## Text Preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    text = re.sub(r"<U\+2019>|<U\+2018>", "\'", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " had ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"n\'t", " not", text)
    text= re.sub(r"<U\+.+>",' ',text)
    text = re.sub(r"[\!\?\.\,\+\-\$\%\^\>\<\=\:\;\*\(\)\{\}\[\]\/\~\&\|]", " ", text)
    return text


In [ ]:
words=[]
def create_corpus(messages):
  corpus=[]
  for i in range(len(messages)):
    review=clean_text(messages['title'][i])
    review.lower()
    review=review.split()
    review=[word for word in review if word.isalpha()]
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    for word in review:
      if word not in words:
        words.append(word)
    review=' '.join(review).strip()
    if len(review)==0:
      review='no text'
    corpus.append(review)
  return corpus 


In [ ]:
corpus_train=create_corpus(messages_train)
corpus_test=create_corpus(messages_test)
corpus_val=create_corpus(messages_val)
len(words)

5976

In [ ]:
len(corpus_train),len(corpus_test),len(corpus_val)

(2836, 997, 526)

In [ ]:
corpus_train[0],corpus_test[0],corpus_val[0]

('At donald properti showcas brand presid elect',
 'onli hillari clinton Is prepar nuclear threat wsj',
 'magazin say six peopl corrobor claim trump assault her')

One Hot Encoding

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [ ]:
voc_size=6000
onehot_repr_train=[one_hot(words,voc_size) for words in corpus_train]
onehot_repr_test=[one_hot(words,voc_size) for words in corpus_test]
onehot_repr_val=[one_hot(words,voc_size) for words in corpus_val]
len(onehot_repr_train),len(onehot_repr_test),len(onehot_repr_val)

(2836, 997, 526)

In [ ]:
onehot_repr_val

[[4780, 4831, 4487, 103, 3719, 1579, 4261, 3909, 3502],
 [3341, 4261, 4008, 1613, 2925, 3149, 5190, 5191, 2525, 1400, 3194],
 [4587, 1540, 4436, 1795, 4453, 4261, 4049],
 [3278, 103, 4780, 3094, 3851, 1988, 4261],
 [4984, 1468, 895, 534, 2046, 868],
 [1654, 4178, 4261, 3225],
 [567, 3017, 1916, 2175, 4261, 2838, 5065],
 [3341, 1400, 2088, 4561, 1502, 490],
 [3537, 4261, 3770, 1220, 5041, 1736, 771, 1255, 1695],
 [2771, 4721, 1009],
 [4905, 743, 4273, 5642, 5191, 867, 5585, 1009],
 [1113, 5904, 1009],
 [5291, 4948, 76, 4123, 389, 3341, 4261, 1888, 1579, 1009],
 [507, 2313, 2249, 743, 4209, 4862, 3039, 5191, 1009],
 [4462, 4555, 363, 1793, 1862, 3130, 5973, 314, 1009],
 [743, 4452, 4779, 4149, 4742, 1009],
 [4261, 1778, 1484, 3006, 4952, 3759, 1009],
 [76, 359, 4585, 2719, 3341, 4261, 787, 908, 2627, 1009],
 [3900, 4609, 1387, 1009],
 [567, 1161, 5128, 3199, 4585, 2719, 1778, 2682, 5834, 1009],
 [4261, 3817, 3466, 768, 5585, 1220, 5181, 1009],
 [5135, 451, 2139, 4000, 797, 1766, 364, 334

In [ ]:
onehot_repr_train[0],corpus_train[0]

([1720, 3341, 5271, 3790, 344, 3309, 4775],
 'At donald properti showcas brand presid elect')

In [ ]:
max_sent_len=0
for sent in onehot_repr_train:
  l=len(sent)
  if max_sent_len<l:
    max_sent_len=l
for sent in onehot_repr_test:
  l=len(sent)
  if max_sent_len<l:
    max_sent_len=l
for sent in onehot_repr_val:
  l=len(sent)
  if max_sent_len<l:
    max_sent_len=lmax_sent_len
max_sent_len

37

In [ ]:
embedded_docs_train=pad_sequences(onehot_repr_train,padding='pre',maxlen=max_sent_len)
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=max_sent_len)
embedded_docs_val=pad_sequences(onehot_repr_val,padding='pre',maxlen=max_sent_len)
embedded_docs_train

array([[   0,    0,    0, ...,  344, 3309, 4775],
       [   0,    0,    0, ...,   53, 3610, 3466],
       [   0,    0,    0, ..., 1795, 4554, 4196],
       ...,
       [   0,    0,    0, ...,    0, 3412, 5077],
       [   0,    0,    0, ...,    0, 3412, 5077],
       [   0,    0,    0, ...,    0, 3412, 5077]], dtype=int32)

In [ ]:
embedded_docs_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 1720, 3341, 5271,
       3790,  344, 3309, 4775], dtype=int32)

#Model

In [ ]:
from keras.layers import Embedding,LSTM,Bidirectional,Dense,Dropout
from keras.models import Sequential
import keras

In [ ]:
embedded_dim=40#dimension of embedding layer
model=Sequential()
model.add(Embedding(voc_size,embedded_dim,input_length=max_sent_len))
model.add(Bidirectional(LSTM(80)))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 37, 40)            240000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 160)               77440     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 317,601
Trainable params: 317,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint_cb=keras.callbacks.ModelCheckpoint("/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5",save_best_only=True,monitor='val_accuracy',mode='auto',verbose=1)

In [ ]:
model.fit(embedded_docs_train,y_train,validation_data=(embedded_docs_val,y_val),epochs=10,batch_size=128,callbacks=[checkpoint_cb])

Epoch 1/10
23/23 [==============================] - 6s 145ms/step - loss: 0.6831 - accuracy: 0.5561 - val_loss: 0.6834 - val_accuracy: 0.4430

Epoch 00001: val_accuracy improved from -inf to 0.44297, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5
Epoch 2/10
23/23 [==============================] - 3s 112ms/step - loss: 0.6152 - accuracy: 0.6034 - val_loss: 0.4762 - val_accuracy: 0.8859

Epoch 00002: val_accuracy improved from 0.44297 to 0.88593, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5
Epoch 3/10
23/23 [==============================] - 3s 110ms/step - loss: 0.4359 - accuracy: 0.8556 - val_loss: 0.3920 - val_accuracy: 0.9125

Epoch 00003: val_accuracy improved from 0.88593 to 0.91255, saving model to /content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5
Epoch 4/10
23/23 [==============================] - 3s 109ms/step - loss: 0.

In [ ]:
model=keras.models.load_model('/content/drive/My Drive/Deep Learning/FakeNewsFinalDataset/CSV/BiLSTM_Best_Main_Model1.h5')

In [ ]:
y_pred_val=model.predict_classes(embedded_docs_val)
y_pred_val

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
accuracy=accuracy_score(y_val,y_pred_val)
accuracy

0.9125475285171103

In [ ]:
print(classification_report(y_val,y_pred_val))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92       293
           1       0.86      0.96      0.91       233

    accuracy                           0.91       526
   macro avg       0.91      0.92      0.91       526
weighted avg       0.92      0.91      0.91       526

